In [4]:
pip install -r requirements.txt

  Obtaining dependency information for vosk from https://files.pythonhosted.org/packages/c0/4c/deb0861f7da9696f8a255f1731bb73e9412cca29c4b3888a3fcb2a930a59/vosk-0.3.45-py3-none-win_amd64.whl.metadata
  Obtaining dependency information for geocoder from https://files.pythonhosted.org/packages/4f/6b/13166c909ad2f2d76b929a4227c952630ebaf0d729f6317eb09cbceccbab/geocoder-1.38.1-py2.py3-none-any.whl.metadata
  Using cached srt-3.5.3-py3-none-any.whl
  Obtaining dependency information for websockets from https://files.pythonhosted.org/packages/98/93/e36c73f78400a65f5e236cd376713c34182e6663f6889cd45a4a04d8f203/websockets-15.0.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for ratelim from https://files.pythonhosted.org/packages/f2/98/7e6d147fd16a10a5f821db6e25f192265d6ecca3d82957a4fdd592cad49c/ratelim-0.1.6-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.0 MB 9.6 MB/s

In [18]:
import speech_recognition as sr
import tempfile
import os
import time
from datetime import datetime
from ai_safety_system.keyword_detection import EMERGENCY_KEYWORDS
from ai_safety_system.scream_detection import predict
import requests
from datetime import datetime
import geocoder
import sys
import io
if hasattr(sys.stdout, 'buffer'):
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')


In [19]:
BASE_FOLDER = "saved_audio"
os.makedirs(BASE_FOLDER, exist_ok=True)


In [20]:
def get_datetime():
    now = datetime.now()
    return now.strftime("%Y-%m-%d"), now.strftime("%H-%M-%S")

In [21]:
def recognize_keywords_from_audio(audio_data):
    recognizer = sr.Recognizer()
    try:
        text = recognizer.recognize_google(audio_data)
        print(f"🗣️ Detected Speech: '{text}'")
        for keyword in EMERGENCY_KEYWORDS:
            if keyword.lower() in text.lower():
                print(f"🔑 Emergency keyword detected: '{keyword}'")
                return True
        return False
    except sr.UnknownValueError:
        print("⚠️ Could not understand audio.")
    except sr.RequestError as e:
        print(f"🔌 API error: {e}")
    return False

In [22]:
def trigger_flask_alert(reason_list):
    from datetime import datetime
    import geocoder

    location = "Location not available"
    try:
        g = geocoder.ip('me')
        if g.ok:
            lat, lng = g.latlng
            location = f"https://maps.google.com/?q={lat},{lng}"
    except:
        pass

    data = {
        "reason": ", ".join(reason_list),
        "location": location,
        "timestamp": datetime.now().isoformat()
    }

    try:
        response = requests.post("http://127.0.0.1:5000/send_alert", json=data)
        print(f"📡 API response: {response.json()}")
    except Exception as e:
        print(f"❌ Error sending to Flask API: {e}")

In [23]:
def trigger_flask_alert(reason_list):
    from datetime import datetime
    import geocoder

    location = "Location not available"
    try:
        g = geocoder.ip('me')
        if g.ok:
            lat, lng = g.latlng
            location = f"https://maps.google.com/?q={lat},{lng}"
    except:
        pass

    data = {
        "reason": ", ".join(reason_list),
        "location": location,
        "timestamp": datetime.now().isoformat()
    }

    try:
        response = requests.post("http://127.0.0.1:5000/send_alert", json=data)
        print(f"📡 API response: {response.json()}")
    except Exception as e:
        print(f"❌ Error sending to Flask API: {e}")

In [24]:
def start_alerter():
    print("\n✅ Alerter Activated! Listening in real-time...")
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        recognizer.adjust_for_ambient_noise(source)
        while True:
            try:
                print("🎤 Listening...")
                audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)

                # Step 1: Keyword detection
                keyword_detected = recognize_keywords_from_audio(audio)

                # Step 2: Save audio temporarily for scream detection
                with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_audio:
                    temp_audio_path = temp_audio.name
                    with open(temp_audio_path, "wb") as f:
                        f.write(audio.get_wav_data())

                # Step 3: Scream detection
                scream_detected = predict(temp_audio_path)

                # Step 4: Save only if alert
                if keyword_detected or scream_detected:
                    print("\n🚨 EMERGENCY TRIGGERED!")
                    reasons = []
                    if keyword_detected:
                        print("🔑 Reason: Keyword")
                        reasons.append("keyword detected")
                    if scream_detected:
                        print("📢 Reason: Scream")
                        reasons.append("scream detected")

                    date_str, time_str = get_datetime()
                    date_folder = os.path.join(BASE_FOLDER, date_str)
                    os.makedirs(date_folder, exist_ok=True)

                    filename = os.path.join(date_folder, f"{time_str}.wav")
                    os.rename(temp_audio_path, filename)
                    print(f"💾 Alert clip saved: {filename}")
                    # 🚀 Send alerts
                    trigger_flask_alert(reasons)

                    time.sleep(3)
                else:
                    os.remove(temp_audio_path)  # Clean up if no alert
                    print("✅ No emergency.\n")

            except KeyboardInterrupt:
                print("\n🛑 Alerter deactivated by user.")
                break
            except Exception as e:
                print(f"❌ Error: {e}")
                continue

In [25]:
if __name__ == "__main__":
    input("🔘 Press Enter to activate the ALERTER (or Ctrl+C to cancel)...")
    start_alerter()


✅ Alerter Activated! Listening in real-time...
🎤 Listening...
⚠️ Could not understand audio.
🎧 Prediction: NON-SCREAM
📈 Confidence: 99.99%
✅ No emergency.

🎤 Listening...
🗣️ Detected Speech: 'hello hello'
🎧 Prediction: NON-SCREAM
📈 Confidence: 97.90%
✅ No emergency.

🎤 Listening...
🗣️ Detected Speech: 'hello hello help'
🔑 Emergency keyword detected: 'help'
🎧 Prediction: NON-SCREAM
📈 Confidence: 98.87%

🚨 EMERGENCY TRIGGERED!
🔑 Reason: Keyword
💾 Alert clip saved: saved_audio\2025-04-18\07-01-18.wav
📡 API response: {'sms_sent': False, 'status': 'alert_received'}
🎤 Listening...
⚠️ Could not understand audio.
🎧 Prediction: NON-SCREAM
📈 Confidence: 99.99%
✅ No emergency.

🎤 Listening...
⚠️ Could not understand audio.
🎧 Prediction: NON-SCREAM
📈 Confidence: 100.00%
✅ No emergency.

🎤 Listening...
⚠️ Could not understand audio.
🎧 Prediction: NON-SCREAM
📈 Confidence: 99.98%
✅ No emergency.

🎤 Listening...

🛑 Alerter deactivated by user.
